# Evaluate API pipeline

This notebook run an evaluation script based on the Rouge Metric to compute the eficience of the API pre-trained pipeline to generate and translate captions.

In [10]:
predict = False
max_qnt = 5000

## Imports

In [11]:
!pip install evaluate pandas
!pip install transformers torch
!pip install rouge_score


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
from evaluate import load
import pandas as pd
import requests
import base64
import os
import json
from transformers import AutoTokenizer 
from transformers import AutoModel
import torch
import nltk
nltk.download("punkt")

/home/mariaeas/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mariaeas/.local/lib/python3.11/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
[nltk_data] Downloading package punkt to /home/mariaeas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Build predictions dataset

In [14]:
test_set = "test.csv"
test_df = pd.read_csv(test_set)

In [5]:
test_df.shape

(12612, 3)

In [6]:
images = list(test_df.image)

In [7]:
try:
    ACCESS_TOKEN = "9c0503cad7176e02bac7d2b411c592db"
    URL = "http://localhost:5000/api/captioner"


    # CALL THE LOGIN ROUTE
    r = requests.post("http://localhost:5000/api/login", json={
        "email": "superadmin@localhost.com",
        "password": ACCESS_TOKEN, 
        "remember": True
    })

    response = json.loads(r.content)
    token = response["data"]["token"]
    print("Login successfully did.")
except Exception as e:
    print("Not possible to connect with the API.")


Login successfully did.


In [8]:
old_pred = None
if os.path.exists("predictions_new.csv"):
    old_pred = pd.read_csv("predictions_new.csv")

if predict:

    pred_df = {
        "image": [],
        "prediction": [],
        "caption": []
    }
    
    if old_pred:
        pred_df = old_pred.to_dict()
    
    for i in range(test_df.shape[0]):

        if old_pred and image in old_pred['image']:
            continue

        row = test_df.iloc[i]
        image = row['image']
        caption = row['caption']

        image_path = f"images/63k-subset/{image}"
        with open(image_path, 'rb') as image_file:
            base64_bytes = base64.b64encode(image_file.read())
            base64_string = "data:image/jpg;base64," + base64_bytes.decode()

            r = json.loads(requests.post(URL, json={
                "images_srcs": [base64_string]
            }, headers={
                "Authorization": f"Bearer {token}"
            }).content)

            pred_df["prediction"].append(r["0"])
            pred_df["image"].append(image)
            pred_df["caption"].append(caption)

            df = pd.DataFrame(pred_df)
            df.to_csv("predictions_new.csv")
            print("file", image, 'done', i)

file i-00098484.jpg done 0
file i-00098503.jpg done 1
file i-00098515.jpg done 2
file i-00098481.jpg done 3
file i-00098508.jpg done 4
file i-00098482.jpg done 5
file i-00098488.jpg done 6
file i-00098498.jpg done 7
file i-00098500.jpg done 8
file i-00098504.jpg done 9
file i-00098516.jpg done 10
file i-00098506.jpg done 11
file i-00098518.jpg done 12
file i-00098501.jpg done 13
file i-00098509.jpg done 14
file i-00098510.jpg done 15
file i-00098511.jpg done 16
file i-00098494.jpg done 17
file i-00098497.jpg done 18
file i-00098496.jpg done 19
file i-00098479.jpg done 20
file i-00098480.jpg done 21
file i-00098483.jpg done 22
file i-00098485.jpg done 23
file i-00098486.jpg done 24
file i-00098487.jpg done 25
file i-00098489.jpg done 26
file i-00098490.jpg done 27
file i-00098491.jpg done 28
file i-00098492.jpg done 29
file i-00098493.jpg done 30
file i-00098495.jpg done 31
file i-00098499.jpg done 32
file i-00098502.jpg done 33
file i-00098505.jpg done 34
file i-00098507.jpg done 35
fi

KeyboardInterrupt: 

In [13]:
df = pd.read_csv("predictions_new.csv")
df.head()
# joint = df.merge(test_df, on="image").dropna()
# joint = joint[["image", "prediction", "caption"]]
# joint.head()

,Unnamed: 0,image,prediction,caption
0,0,i-00098484.jpg,Uma pessoa que mantém um surfboard em um corpo...,i-00098484.jpg
1,1,i-00098503.jpg,Uma mão que mantém um pedaço de papel no topo ...,i-00098503.jpg
2,2,i-00098515.jpg,uma colagem de fotos que mostram diferentes ti...,i-00098515.jpg
3,3,i-00098481.jpg,Um urso tedi usando um chapéu e óculos de sol,i-00098481.jpg
4,4,i-00098508.jpg,Uma mulher e um homem postando para uma foto,i-00098508.jpg


In [17]:
sub_test = test_df[:df.shape[0]]


In [19]:
sub_df = df[['image', 'prediction']]

In [22]:
df_final = sub_test.merge(sub_df, on='image')[['image', 'caption', 'prediction']]
df_final.head()

,image,caption,prediction
0,i-00098484.jpg,"capa do livro 71 leões, título escrito na co...",Uma pessoa que mantém um surfboard em um corpo...
1,i-00098503.jpg,imagem em Braille de uma página do livro O Peq...,Uma mão que mantém um pedaço de papel no topo ...
2,i-00098515.jpg,foto de uma tabela com as informações dos curs...,uma colagem de fotos que mostram diferentes ti...
3,i-00098481.jpg,"coelhinho branco de orelhas rosadas, usando óc...",Um urso tedi usando um chapéu e óculos de sol
4,i-00098508.jpg,"Professora Adriana e a aluna Celinha, fantasia...",Uma mulher e um homem postando para uma foto


In [5]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mariaeas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
df_final = pd.read_csv('prediction_reference.csv')[:2133]
df_final.head()

,Unnamed: 0,image,caption,prediction
0,0,i-00098484.jpg,capa livro 71 leões título escrito cor amarela...,pessoa mantém surfboard corpo água
1,1,i-00098503.jpg,imagem braille página livro pequeno príncipe m...,mão mantém pedaço papel topo mesa
2,2,i-00098515.jpg,foto tabela informações cursos formação oferec...,colagem fotos mostram diferentes tipos disposi...
3,3,i-00098481.jpg,coelhinho branco orelhas rosadas usando óculos...,urso tedi usando chapéu óculos sol
4,4,i-00098508.jpg,professora adriana aluna celinha fantasiadas r...,mulher homem postando foto


In [6]:
sw = stopwords.words('portuguese')
sw.extend(['.', ":", "!", "?", "\\", "/", "*", "-", ",", "#", "@"])
def clean_text(text):
    tokens = nltk.word_tokenize(text, language="portuguese")
    filtered_tokens = list(filter(lambda w: w not in sw, tokens))

    return " ".join(filtered_tokens).lower()

In [7]:
df_final['caption'] = df_final['caption'].apply(clean_text)
df_final['prediction'] = df_final['prediction'].apply(clean_text)

df_final.head()

,Unnamed: 0,image,caption,prediction
0,0,i-00098484.jpg,capa livro 71 leões título escrito cor amarela...,pessoa mantém surfboard corpo água
1,1,i-00098503.jpg,imagem braille página livro pequeno príncipe m...,mão mantém pedaço papel topo mesa
2,2,i-00098515.jpg,foto tabela informações cursos formação oferec...,colagem fotos mostram diferentes tipos disposi...
3,3,i-00098481.jpg,coelhinho branco orelhas rosadas usando óculos...,urso tedi usando chapéu óculos sol
4,4,i-00098508.jpg,professora adriana aluna celinha fantasiadas r...,mulher homem postando foto


# Calculate metrics

In [8]:
candidates = list(df_final.prediction)
mult_references = list(map(lambda x: [x], list(df_final.caption)))

## Calculate BLEU

In [9]:
metric_bleu = load('bleu')
result_bleu = metric_bleu.compute(predictions=candidates, references=mult_references)
result_bleu

{'bleu': 0.0,
 'precisions': [0.11457306251232499,
  0.009239605443875639,
  0.0008509189925119128,
  0.0],
 'brevity_penalty': 0.01072264565596614,
 'length_ratio': 0.1806555040969006,
 'translation_length': 10142,
 'reference_length': 56140}

## Calculate ROUGE

In [10]:
metric_rouge = load('rouge')
result_rouge = metric_rouge.compute(predictions=candidates, references=mult_references)
result_rouge

{'rouge1': 0.05203651781532252,
 'rouge2': 0.006472529862411563,
 'rougeL': 0.04806922820154137,
 'rougeLsum': 0.04784850938481548}

## Calculate Cosine Similarity

In [11]:
model = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=True, padding='max_length', max_len=500)

In [12]:
from torch.nn.functional import cosine_similarity

In [13]:
mean_cosine_similarity = 0
for i in range(df_final.shape[0]):
    row = df_final.iloc[i]
    
    sentence1 = str(row['caption'])
    sentence2 = str(row['prediction'])

    # Tokenize and encode sentences
    inputs1 = tokenizer(sentence1, return_tensors='pt', truncation=True, padding=True)
    inputs2 = tokenizer(sentence2, return_tensors='pt', truncation=True, padding=True)

    # Encode sentences
    with torch.no_grad():
        outputs1 = model(**inputs1)
        outputs2 = model(**inputs2)

    # Extract embeddings
    embeddings1 = outputs1.last_hidden_state[:, 0, :]
    embeddings2 = outputs2.last_hidden_state[:, 0, :]

    # Calculate cosine similarity
    similarity_score = cosine_similarity(embeddings1, embeddings2).item()
    mean_cosine_similarity += similarity_score
    print(f"file: {row['image']} | similarity = {similarity_score}")

file: i-00098484.jpg | similarity = 0.6129022240638733
file: i-00098503.jpg | similarity = 0.7084088921546936
file: i-00098515.jpg | similarity = 0.6580905318260193
file: i-00098481.jpg | similarity = 0.7685728073120117
file: i-00098508.jpg | similarity = 0.5792534351348877
file: i-00098482.jpg | similarity = 0.49423378705978394
file: i-00098488.jpg | similarity = 0.6060400009155273
file: i-00098498.jpg | similarity = 0.6066944599151611
file: i-00098500.jpg | similarity = 0.6130473613739014
file: i-00098504.jpg | similarity = 0.5594409704208374
file: i-00098516.jpg | similarity = 0.6492603421211243
file: i-00098506.jpg | similarity = 0.8149133920669556
file: i-00098518.jpg | similarity = 0.6686025261878967
file: i-00098501.jpg | similarity = 0.63606196641922
file: i-00098509.jpg | similarity = 0.7441084384918213
file: i-00098510.jpg | similarity = 0.7475065588951111
file: i-00098511.jpg | similarity = 0.6704415082931519
file: i-00098494.jpg | similarity = 0.6517284512519836
file: i-000

In [14]:
mean_cosine_similarity /= df_final.shape[0]
mean_cosine_similarity

0.6876932013861964

# METEOR

In [15]:
meteor = load('meteor')
result_meteor = meteor.compute(predictions=candidates, references=mult_references)
result_meteor

[nltk_data] Downloading package wordnet to /home/mariaeas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/mariaeas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/mariaeas/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


{'meteor': 0.018625489132563643}

# CIDeR

In [16]:
from aac_metrics import evaluate

cider, _ = evaluate(candidates, mult_references, metrics="cider_d")
float(cider['cider_d'])

NameError: name 'evaluate' is not defined

# Metrics Overview

In [ ]:
metrics = {
    "metric": ["BLEU1", "BLEU2","BLEU3","BLEU4", "ROUGE1", "ROUGE2", "ROUGEL", "METEOR", "CIDEr-D", "COSINE SIMILARITY"],
    "value": [result_bleu['precisions'][0], result_bleu['precisions'][1],result_bleu['precisions'][2], result_bleu['precisions'][3], result_rouge['rouge1'], result_rouge['rouge2'], result_rouge['rougeL'], result_meteor['meteor'], float(cider['cider_d']), mean_cosine_similarity]
}

df_metrics = pd.DataFrame(metrics)
df_metrics

In [14]:
df_metrics = pd.DataFrame(metrics)
df_metrics

,metric,value
0,BLEU1,0.114573
1,BLEU2,0.009240
2,ROUGE1,0.051972
3,ROUGE2,0.006436
4,ROUGEL,0.048070
5,COSINE SIMILARITY,0.687693


In [15]:
df_metrics.to_csv("metrics_api.csv")

In [35]:
df_final.to_csv("prediction_reference.csv")